<a href="https://colab.research.google.com/github/hansjang/Py_Radar/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0Part3_P186.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(P.186) 1. 실습 A to Z : Moter Trend Car Road Tests

**(1) 문제 : 32가지 자동차의 연료 소비를 포함한 데이터로 차량모델이 미국산인지 타지역인지를 분류하는 분석**

In [ ]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

**(2) 사전 준비**



In [2]:
import pandas as pd
import numpy as np
X_test = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/mpg_X_test.csv")
X_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/mpg_X_train.csv")
y_train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part2/mpg_y_train.csv")

**(3) 데이터 전처리**

In [5]:
print(X_train.info())
print(X_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          278 non-null    object 
 1   mpg           278 non-null    float64
 2   cylinders     278 non-null    int64  
 3   displacement  278 non-null    float64
 4   horsepower    274 non-null    float64
 5   weight        278 non-null    int64  
 6   acceleration  278 non-null    float64
 7   model_year    278 non-null    int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 17.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          120 non-null    object 
 1   mpg           120 non-null    float64
 2   cylinders     120 non-null    int64  
 3   displacement  120 non-null    float64
 4   horsepower    118 non-null    f

**[결측치 처리]**
X_train 및 X_test에는 8개의 컬럼이 있으며, horsepower 변수에 X_train의 경우 4개, X_test의 경우 2개의 결측치가 있음을 확인하였다. name 컬럼을 제외한 나머지 컬럼들은 모두 float64, 혹은 int64로서 수치형이므로 별도의 데이터 타입 변경 작업은 필요없을 것으로 보인다. 따라서 horsepower에 대한 결측치부터 처리한다. 결측치는 평균값 대치로 처리했다.

In [6]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train[['horsepower']] = imputer.fit_transform(X_train[['horsepower']])
X_test[['horsepower']] = imputer.fit_transform(X_test[['horsepower']])

In [ ]:
print(X_train.info())
print(X_test.info())

In [16]:
print(round(X_train.describe(),2))

          mpg  cylinders  displacement  horsepower   weight  acceleration  \
count  278.00     278.00        278.00      278.00   278.00        278.00   
mean    23.73       5.37        189.99      103.38  2948.46         15.58   
std      7.65       1.68        105.47       38.70   862.95          2.75   
min     10.00       3.00         68.00       46.00  1613.00          8.00   
25%     18.00       4.00         98.00       75.00  2206.25         14.00   
50%     23.00       4.00        140.50       90.50  2737.50         15.50   
75%     29.00       6.00        258.00      118.75  3560.00         17.00   
max     46.60       8.00        455.00      230.00  5140.00         24.80   

       model_year  
count      278.00  
mean        76.06  
std          3.61  
min         70.00  
25%         73.00  
50%         76.00  
75%         79.00  
max         82.00  


**[변수 형태별 리스트 저장]**
변수 형태별로 분류하여 리스트로 저장한다. model_year의 경우 명목형 변수로 생각할 수 있지만, 여기서는 나중에 생산된 모델의 성능이 더 우수할 것이라는 전제를 바탕으로 수치형 변수로 사용하고자 한다. 문자형 변수인 name은 분석에 영향을 미치지 않으므로 제거한다.  

In [9]:
COL_DEL = ['name']
COL_NUM = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']
COL_CAT = []
COL_Y = ['isUSA']

X_train = X_train.iloc[:, 1:]
X_test = X_test.iloc[:, 1:]

In [ ]:
print(X_train.info())
print(X_test.info())

**(4) 데이터 모형 구축**



In [11]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.3)

**[스케일링 처리]**
수치형 변수들의 범위 차이에 따른 영향력이 왜곡되지 않도록 동일한 조건에서 분석하기 위해 스케일링을 진행한다.

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

In [15]:
print(round(X_tr.describe(), 2))
print(round(X_val.describe(), 2))
print(round(X_test.describe(), 2))

          mpg  cylinders  displacement  horsepower  weight  acceleration  \
count  194.00     194.00        194.00      194.00  194.00        194.00   
mean    -0.00       0.00         -0.00        0.00   -0.00          0.00   
std      1.00       1.00          1.00        1.00    1.00          1.00   
min     -1.86      -1.39         -1.13       -1.48   -1.53         -2.81   
25%     -0.80      -0.78         -0.84       -0.70   -0.85         -0.62   
50%     -0.00      -0.78         -0.43       -0.31   -0.25         -0.07   
75%      0.71       0.45          0.65        0.23    0.61          0.54   
max      2.98       1.67          2.66        3.30    2.63          3.32   

       model_year  
count      194.00  
mean         0.00  
std          1.00  
min         -1.65  
25%         -0.84  
50%         -0.03  
75%          0.78  
max          1.59  
         mpg  cylinders  displacement  horsepower  weight  acceleration  \
count  84.00      84.00         84.00       84.00   84.00   

**[분류모형 학습]**
sklearn 패키지를 이용하여 KNN, 의사결정나무 모델을 만든다.

In [19]:
from sklearn.neighbors import KNeighborsClassifier
modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr, y_tr.values.ravel())
# ravel() : 1차원 배열의 타깃 벡터로 변환

from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth = 10)
modelDT.fit(X_tr, y_tr)

DecisionTreeClassifier(max_depth=10)

**(5) 데이터 모형 평가**

In [20]:
y_val_pred = modelKNN.predict(X_val)
# predict() : 예측값을 구한다

y_val_pred_probaKNN = modelKNN.predict_proba(X_val)
y_val_pred_probaDT = modelDT.predict_proba(X_val)
# predict_proba() : 예측확률값을 구한다

In [23]:
from sklearn.metrics import roc_auc_score

scoreKNN = roc_auc_score(y_val, y_val_pred_probaKNN[:, 1])
scoreDT = roc_auc_score(y_val, y_val_pred_probaDT[:, 1])

print(scoreKNN, scoreDT)

0.937962962962963 0.862962962962963


AUROC 점수를 구한 결과, KNN모형이 DT모형보다 높은 점수를 기록했다. KNN 모형에서 여러 하이퍼파라미터를 학습해 최적의 모형을 도출하고자 한다.

In [28]:
best_model = None
best_score = 0

for i in range(2, 10) :
  model = KNeighborsClassifier(n_neighbors=i, metric='euclidean')
  model.fit(X_tr, y_tr.values.ravel())
  y_val_pred_proba = model.predict_proba(X_val)
  score = roc_auc_score(y_val, y_val_pred_proba[:, 1])
  print(i, "개의 이웃 확인 : ", score)
  if best_score <= score :
    best_model = model

2 개의 이웃 확인 :  0.9182098765432098
3 개의 이웃 확인 :  0.9148148148148147
4 개의 이웃 확인 :  0.9293209876543209
5 개의 이웃 확인 :  0.937962962962963
6 개의 이웃 확인 :  0.9305555555555556
7 개의 이웃 확인 :  0.9345679012345679
8 개의 이웃 확인 :  0.9354938271604938
9 개의 이웃 확인 :  0.9530864197530864


KNN 모형에서 하이퍼파라미터인 n_neighbors 값을 2부터 9까지 차례로 넣어본 결과 n_neighbors = 9 인 모델이 best model로 저장된다.

In [29]:
print(best_model)

KNeighborsClassifier(metric='euclidean', n_neighbors=9)


In [ ]:
print(best_model.predict_proba(X_test))

In [31]:
pred = best_model.predict_proba(X_test)[:,1]
print(pred)

[0.22222222 0.77777778 1.         1.         0.22222222 0.33333333
 0.44444444 0.44444444 0.44444444 0.55555556 1.         0.55555556
 1.         0.44444444 0.22222222 0.33333333 1.         1.
 0.22222222 1.         1.         0.88888889 1.         1.
 1.         0.55555556 0.33333333 1.         1.         0.44444444
 0.         0.11111111 0.33333333 1.         1.         0.55555556
 0.55555556 1.         0.33333333 0.55555556 1.         0.33333333
 0.11111111 0.55555556 0.22222222 0.44444444 0.33333333 1.
 0.33333333 1.         0.         0.22222222 1.         0.11111111
 1.         1.         1.         0.77777778 1.         0.33333333
 1.         0.77777778 1.         1.         0.88888889 1.
 1.         0.33333333 0.66666667 0.11111111 0.88888889 1.
 0.44444444 0.11111111 1.         0.44444444 1.         0.44444444
 0.44444444 1.         1.         1.         0.         1.
 1.         0.88888889 0.22222222 0.11111111 0.22222222 1.
 1.         1.         1.         1.         1.    

**[최종]**
결과물을 파일로 저장한다.

In [32]:
pd.DataFrame({'isUSA' : pred}).to_csv('./240805_P186.csv')